In [1]:
import pandas as pd
import time
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score

In [2]:
train = pd.read_csv('/kaggle/input/Cascade_cup/train_age_dataset.csv')
test = pd.read_csv('/kaggle/input/Cascade_cup/test_age_dataset.csv')
sample = pd.read_csv('/kaggle/input/Cascade_cup/sample_submission.csv')

In [3]:
train

,Unnamed: 0,userId,tier,gender,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,...,content_views,num_of_comments,weekends_trails_watched_per_day,weekdays_trails_watched_per_day,slot1_trails_watched_per_day,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group
0,265153,48958844,2,1,0.000000,0.000,0.00,0,0.0,0.0,...,0.200000,0.000000,0.041667,0.025000,0.000000,0.000000,0.175000,0.033333,0.000000,1
1,405231,51100441,2,2,0.000000,0.000,0.00,0,0.0,0.0,...,0.093220,0.000000,0.012712,0.018644,0.000000,0.084746,0.000000,0.033898,82.500000,2
2,57867,6887426,2,1,0.000000,0.000,0.00,0,0.0,0.0,...,0.002786,0.000000,0.000000,0.000557,0.000000,0.002786,0.000000,0.000000,0.000000,1
3,272618,50742404,2,1,0.000000,0.000,0.00,0,0.0,0.0,...,0.008403,0.000000,0.000000,0.001681,0.000000,0.000000,0.000000,0.008403,0.000000,1
4,251123,45589200,2,2,0.000000,0.000,0.00,0,0.0,0.0,...,0.204918,0.000000,0.000000,0.049180,0.000000,0.008197,0.057377,0.180328,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488872,535266,38983571,2,2,0.000000,0.000,0.00,0,0.0,0.0,...,0.064000,0.000000,0.012000,0.006400,0.000000,0.008000,0.000000,0.048000,195.000000,1
488873,36706,53264639,1,1,0.017241,0.000,0.00,0,0.0,0.0,...,2.043103,0.017241,0.301724,0.339655,0.068966,0.922414,0.137931,1.172414,188.742857,4
488874,31570,44693578,1,1,0.229508,1.625,2.25,0,0.0,0.0,...,0.409836,0.000000,0.000000,0.118033,0.000000,0.049180,0.237705,0.303279,184.880000,4
488875,239209,42760145,2,1,0.000000,0.000,0.00,0,0.0,0.0,...,0.113821,0.000000,0.000000,0.030894,0.000000,0.024390,0.130081,0.000000,0.000000,1


In [4]:
train = pd.get_dummies(train, columns=['tier','gender'])
test = pd.get_dummies(test, columns=['tier','gender'])

# REMEMBER TO ADD 1 IN THE END

train['age_group'] = train['age_group']-1

In [5]:
main_feature_cols = [col for col in train.columns.tolist() if col not in ['age_group']]

In [6]:
train

,Unnamed: 0,userId,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,...,slot2_trails_watched_per_day,slot3_trails_watched_per_day,slot4_trails_watched_per_day,avgt2,age_group,tier_1,tier_2,tier_3,gender_1,gender_2
0,265153,48958844,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.000000,...,0.000000,0.175000,0.033333,0.000000,0,0,1,0,1,0
1,405231,51100441,0.000000,0.000,0.00,0,0.0,0.0,0.076923,0.153846,...,0.084746,0.000000,0.033898,82.500000,1,0,1,0,0,1
2,57867,6887426,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.000000,...,0.002786,0.000000,0.000000,0.000000,0,0,1,0,1,0
3,272618,50742404,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.008403,0.000000,0,0,1,0,1,0
4,251123,45589200,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.000000,...,0.008197,0.057377,0.180328,0.000000,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488872,535266,38983571,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.300000,...,0.008000,0.000000,0.048000,195.000000,0,0,1,0,0,1
488873,36706,53264639,0.017241,0.000,0.00,0,0.0,0.0,0.000000,0.012195,...,0.922414,0.137931,1.172414,188.742857,3,1,0,0,1,0
488874,31570,44693578,0.229508,1.625,2.25,0,0.0,0.0,0.000000,0.094340,...,0.049180,0.237705,0.303279,184.880000,3,1,0,0,1,0
488875,239209,42760145,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.000000,...,0.024390,0.130081,0.000000,0.000000,0,0,1,0,1,0


In [7]:
from sklearn.decomposition import PCA


NUM_PCA = 3
train_features = train.drop('age_group', axis=1)

pca = PCA(n_components=NUM_PCA, random_state=0)
data = train_features.append(test)
pca_fts = pca.fit_transform(data[main_feature_cols]).reshape(-1, NUM_PCA)
pca_cols = []
for i in range(NUM_PCA):
    data[f'pca_{i}'] = pca_fts[:, i]
    pca_cols.append(f'pca_{i}')

train_features = data.iloc[:train.shape[0], :]
test = data.iloc[train.shape[0]:, :]

train = pd.concat([train_features, train['age_group']], axis=1)

In [8]:
train

,Unnamed: 0,userId,following_rate,followers_avg_age,following_avg_age,max_repetitive_punc,num_of_hashtags_per_action,emoji_count_per_action,punctuations_per_action,number_of_words_per_action,...,avgt2,tier_1,tier_2,tier_3,gender_1,gender_2,pca_0,pca_1,pca_2,age_group
0,265153,48958844,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.000000,...,0.000000,0,1,0,1,0,-6.585837e+06,-18423.021887,-18.419889,0
1,405231,51100441,0.000000,0.000,0.00,0,0.0,0.0,0.076923,0.153846,...,82.500000,0,1,0,0,1,-8.727685e+06,117759.739874,46.826853,1
2,57867,6887426,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.000000,...,0.000000,0,1,0,1,0,3.548589e+07,-149191.428065,-451.848957,0
3,272618,50742404,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.000000,...,0.000000,0,1,0,1,0,-8.369407e+06,-14201.868368,-32.360541,0
4,251123,45589200,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.000000,...,0.000000,0,1,0,0,1,-3.216173e+06,-26324.460280,-72.742800,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488872,535266,38983571,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.300000,...,195.000000,0,1,0,0,1,3.388928e+06,269832.050717,10.737846,0
488873,36706,53264639,0.017241,0.000,0.00,0,0.0,0.0,0.000000,0.012195,...,188.742857,1,0,0,1,0,-1.089121e+07,-254700.674622,-94.600345,3
488874,31570,44693578,0.229508,1.625,2.25,0,0.0,0.0,0.000000,0.094340,...,184.880000,1,0,0,1,0,-2.320153e+06,-244248.076255,-184.694046,3
488875,239209,42760145,0.000000,0.000,0.00,0,0.0,0.0,0.000000,0.000000,...,0.000000,0,1,0,1,0,-3.871009e+05,-33093.106640,-102.845293,0


In [9]:
#feature_cols = [col for col in train.columns.tolist() if col not in ['age_group','gender_2', 'tier_1','tier_3',
#                                                                    'num_of_hashtags_per_action','avgDuration','avgCompletion']]

feature_cols = [col for col in train.columns.tolist() if col not in ['age_group']]

target_cols = ['age_group']

In [10]:
from sklearn.model_selection import train_test_split
xtrain, xval, ytrain, yval = train_test_split(train[feature_cols], train[target_cols], test_size=0.2)
clf = xgb.XGBClassifier()
clf.fit(xtrain, ytrain)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [11]:
pd.DataFrame(zip(feature_cols, clf.feature_importances_)).sort_values(by=1)

,0,1
28,gender_2,0.000000
24,tier_1,0.000000
26,tier_3,0.001790
13,avgComments,0.002636
6,num_of_hashtags_per_action,0.002916
12,avgDuration,0.003124
10,avgCompletion,0.003146
21,slot3_trails_watched_per_day,0.003421
20,slot2_trails_watched_per_day,0.003545
17,weekends_trails_watched_per_day,0.003575


In [12]:
# KFOLD
skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

for i, (trn, val) in enumerate(skf.split(train[target_cols], train[target_cols])):
    train.loc[val, 'kfold'] = i
train['kfold'] = train['kfold'].astype(int)

In [13]:
def run_training():
    oof = np.zeros((train.shape[0], 4))
    pred = np.zeros((test.shape[0], 4))
    
    for fold in range(5):
        
        print(f"\nStarting FOLD: {fold}")
        start = time.time()
        

        trn_idx = train['kfold'] != fold
        val_idx = train['kfold'] == fold
        trn = train.loc[trn_idx, :]
        val = train.loc[val_idx, :]

        xtr, ytr = trn[feature_cols].values, trn[target_cols].values
        xval, yval = val[feature_cols].values, val[target_cols].values
        xtest = test[feature_cols].values

        model = xgb.XGBClassifier(n_estimators = 100)
        print("Fitting model ...")
        model.fit(xtr, ytr.reshape(-1,))
        
        print("Calculating training preds ...")
        train_preds = model.predict(xtr)
        print("Calculating validation preds ...")
        val_preds = model.predict(xval)
        print("Training Accuracy Score - ", accuracy_score(ytr, train_preds ))
        print("Training F1 Score - ", f1_score(ytr, train_preds, average='weighted'))
        
        print("Validation Accuracy Score - ", accuracy_score(yval, val_preds))
        print("Validation F1 Score - ", f1_score(yval, val_preds, average='weighted'))
        
        oof[val_idx, :] += model.predict_proba(xval)
        
        # Test preds
        pred += model.predict_proba(xtest)/5
        
        print(f"FOLD {fold} completed in {time.time()-start} seconds")
        
    return oof, pred

In [14]:
oof_xgb, pred_xgb = run_training()   
#100 trees - 79.00                                  
#150 trees - 79.41                                 
#200 trees - 79.79                                  
#300 trees - unregularized               - 80.17                   
#300 trees - reg_alpha=0.1, reg_lambda=2 - 80.23   
#300 trees - reg_alpha=0.2, reg_lambda=5 - 80.24  -- OOF - 80.26 
#300 trees - reg_alpha=1.5, reg_lambda=5 - 80.20   
#400 trees - unregularized               - 80.33
#500 trees - unregularized               - 80.44
#500 tress - reg_alpha=0.5, reg_lambda=5 - 80.53  -- OOF - 80.56 -- LB 80.89 -- overfit to 1st fold
#500 tress - reg_alpha=0.8, reg_lambda=5 - 80.44  -- OOF - 80.53 -- LB 80.97
#500 trees - reg_alpha=0.7, reg_lambda=0 - 80.47 -- OOF - 80.54 -- LB - 81.01 ----> without two features - 81.05
#500 trees - reg_alpha=1.0, reg_lambda=10 - 80.32 -- OOF - 80.15 -- LB - 80.91


Starting FOLD: 0
Fitting model ...
Calculating training preds ...
Calculating validation preds ...
Training Accuracy Score -  0.8226135959764869
Training F1 Score -  0.8170957178584384
Validation Accuracy Score -  0.7942644411716576
Validation F1 Score -  0.7879456287985567
FOLD 0 completed in 210.32081031799316 seconds

Starting FOLD: 1
Fitting model ...
Calculating training preds ...
Calculating validation preds ...
Training Accuracy Score -  0.8204018910716158
Training F1 Score -  0.8149824680842122
Validation Accuracy Score -  0.7942235313369334
Validation F1 Score -  0.7883092840547401
FOLD 1 completed in 209.32959985733032 seconds

Starting FOLD: 2
Fitting model ...
Calculating training preds ...
Calculating validation preds ...
Training Accuracy Score -  0.8221205721269643
Training F1 Score -  0.8167082946605312
Validation Accuracy Score -  0.7954180516491945
Validation F1 Score -  0.789299624750855
FOLD 2 completed in 210.04349660873413 seconds

Starting FOLD: 3
Fitting model 

In [21]:
def find_oof_score(oof):
    predictions = oof.argmax(axis=1)
    print("OOF Accuracy Score - ", accuracy_score(train[target_cols], predictions))
    print("OOF F1 Score - ", f1_score(train[target_cols], predictions, average='weighted'))

In [22]:
find_oof_score(oof_xgb) # 80.26

OOF Accuracy Score -  0.795091607909556
OOF F1 Score -  0.7890089004070117


In [24]:
pred_xgb.shape

(54320, 4)

In [25]:
final_preds = pred_xgb.argmax(axis=1)+1
pred_csv = pd.DataFrame(final_preds.reshape(-1), columns=['prediction'] )
pred_csv

,prediction
0,1
1,1
2,1
3,2
4,1
...,...
54315,1
54316,1
54317,4
54318,1


In [26]:
pred_csv.value_counts()

prediction
1             34419
2              8463
3              8336
4              3102
dtype: int64

In [27]:
pred_csv.to_csv('withpca.csv', index=False)

In [28]:
oof_xgb.shape, pred_xgb.shape

((488877, 4), (54320, 4))

In [30]:
np.save('oof_xgb.npy', oof_xgb)
np.save('pred_xgb.npy', pred_xgb)